In [43]:
import pandas as pd
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [44]:
# Set options
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

data_raw = pd.read_csv("STAT_444_project_data_power_transformed_noNA.csv", low_memory = True)


#data_raw.dropna()
data_raw.reset_index(drop=True, inplace=True)

##train_raw, test_raw = train_test_split(data_raw, test_size=0.1)


split_index = int(0.9 * len(data_raw))  # 80% for training, 20% for testing
train_raw = data_raw[:split_index] 
test_raw = data_raw[split_index:]


#train_x_raw = pd.read_csv("../01-Data/X_train.csv", low_memory = True, index_col=0)
#train_y_raw = pd.read_csv("../01-Data/y_train.csv", low_memory = True, index_col=0)
#test_x_raw = pd.read_csv("../01-Data/X_test.csv", low_memory=True, index_col=0)

df_train = pd.DataFrame(train_raw).drop(["date_1","date_2", "temp2_c", "temp2_min_c",
            "wind_speed50_ave_m_s", "max_generation_mw"], axis=1)
df_test = pd.DataFrame(test_raw).drop(["date_1","date_2", "temp2_c", "temp2_min_c",
            "wind_speed50_ave_m_s", "max_generation_mw"], axis=1)



In [45]:
# Neural Network
import numpy as np
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm

In [46]:
df_z_scaled = df_train.copy()
df_tz = df_test.copy()
# apply normalization techniques 
for column in df_z_scaled.columns:
    df_z_scaled[column] = df_z_scaled[column].astype(float)
    df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / (df_z_scaled[column].std())

for column in df_tz.columns:
    df_tz[column] = df_tz[column].astype(float)
    df_tz[column] = (df_tz[column] -
                           df_tz[column].mean()) / (df_tz[column].std())
    


In [47]:
df_xtrain = df_z_scaled.drop(['total_demand_mw'], axis=1)
df_xtest = df_tz.drop(['total_demand_mw'], axis=1)
df_ytrain = df_z_scaled['total_demand_mw']
df_ytest = df_tz['total_demand_mw']


df_tensor_xtr = torch.tensor(df_xtrain.values, dtype=torch.float32)
df_tensor_xte = torch.tensor(df_xtest.values, dtype=torch.float32)
df_tensor_ytr = torch.tensor(df_ytrain.values, dtype=torch.float32)
df_tensor_yte = torch.tensor(df_ytest.values, dtype=torch.float32)
df_tensor_xtr = torch.nan_to_num(df_tensor_xtr)
df_tensor_xte = torch.nan_to_num(df_tensor_xte )
df_tensor_ytr = torch.nan_to_num(df_tensor_ytr)
df_tensor_yte = torch.nan_to_num(df_tensor_yte )


In [48]:
df_tensor_xtr

tensor([[-1.5872, -1.6987, -1.3900,  ..., -0.3921, -1.8925, -1.1784],
        [-1.4965, -1.6987, -1.3900,  ...,  0.2090, -0.2405, -0.9279],
        [-1.4010, -1.6987, -1.3900,  ...,  0.5878,  0.8182, -0.5870],
        ...,
        [-0.1068,  1.0704,  1.5058,  ..., -0.0843,  0.4492, -0.0071],
        [ 0.0079,  1.0704,  1.5058,  ..., -0.1262, -0.0597, -0.2682],
        [ 0.1234,  1.0704,  1.5058,  ...,  0.0711, -0.0221,  0.4999]])

In [49]:
class dataset(torch.utils.data.Dataset):
    def __init__(self):
        super().__init__()
        self.x = df_tensor_xtr
        self.t = df_tensor_ytr
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.t[idx]
    
    def inputs(self):
        return self.x
    
    def targets(self):
        return self.t

In [50]:
class datasett(torch.utils.data.Dataset):
    def __init__(self):
        super().__init__()
        self.x = df_tensor_xte
        self.t = df_tensor_yte
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.t[idx]
    
    def inputs(self):
        return self.x
    
    def targets(self):
        return self.t

In [51]:
#batch_size_train = 256
#train_dl = torch.utils.data.DataLoader(dataset(), batch_size=batch_size_train, shuffle=True)
test_all = torch.utils.data.DataLoader(datasett(), batch_size=len(df_test), shuffle=True)

In [52]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

In [53]:
class RegressorNN(nn.Module):
    '''
     
    '''
    def __init__(self):
        self.losses = []
        super().__init__()
        
        #===== YOUR CODE HERE =====
        self.seq= nn.Sequential(
            nn.Linear(9, 32),
            nn.Sigmoid(),
            nn.Linear(32, 16),
            nn.Sigmoid(),
            nn.Linear(16, 8),
            nn.Sigmoid(),
            nn.Linear(8, 1))
    
    def forward(self, x):
        #===== YOUR CODE HERE =====
        x = self.seq(x)
        return x  # replace this line

In [54]:
from sklearn.model_selection import KFold

In [55]:
loss_fn = nn.MSELoss()

k_folds = 5
num_epochs = 2000
# Set fixed random number seed
torch.manual_seed(42)


# Define the K-fold Cross Validator
kf = KFold(n_splits=k_folds, shuffle=True)

In [56]:
all_val_losses = []

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset())):
    print(f'Fold {fold+1}')
    
    train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)
    
    train_loader = torch.utils.data.DataLoader(dataset(), batch_size=256, sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(dataset(), batch_size=256, sampler=val_sampler)
    
    net1 = RegressorNN()
    if torch.cuda.is_available():
      net1 = net1.cuda()
    net1.apply(reset_weights)
    optimizer = torch.optim.SGD(net1.parameters(), lr=0.00001, momentum=0.8)
    
    for epoch in tqdm(range(num_epochs)):
        net1.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            predictions = net1(X_batch)
            loss = loss_fn(predictions, y_batch)
            loss.backward()
            optimizer.step()
    
    # Validation
    net1.eval()
    val_losses = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            predictions = net1(X_batch)
            loss = loss_fn(predictions, y_batch)
            val_losses.append(loss.item())
    fold_val_loss = np.mean(val_losses)
    all_val_losses.append(fold_val_loss)
    print(f'Validation Loss for fold {fold+1}: {np.mean(val_losses)}')


mean_cv_loss = np.mean(all_val_losses)
std_cv_loss = np.std(all_val_losses)

print(f'Final CV Loss: {mean_cv_loss}')

Fold 1
Reset trainable parameters of layer = Linear(in_features=9, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


  0%|          | 0/2000 [00:00<?, ?it/s]c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([120])) that is different to the input size (torch.Size([120, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 2000/2000 [00:56<00:00, 35.15it/s]
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([95]))

Validation Loss for fold 1: 0.9846925139427185
Fold 2
Reset trainable parameters of layer = Linear(in_features=9, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


  0%|          | 0/2000 [00:00<?, ?it/s]c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([121])) that is different to the input size (torch.Size([121, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 2000/2000 [00:59<00:00, 33.50it/s]
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([94])) that is different to the input size (torch.Size([94, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Validation Loss for fold 2: 0.8944589495658875
Fold 3
Reset trainable parameters of layer = Linear(in_features=9, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


...

Validation Loss for fold 3: 1.1008175611495972
Fold 4
Reset trainable parameters of layer = Linear(in_features=9, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


100%|██████████| 2000/2000 [01:01<00:00, 32.58it/s]


Validation Loss for fold 4: 0.9762589037418365
Fold 5
Reset trainable parameters of layer = Linear(in_features=9, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


100%|██████████| 2000/2000 [01:00<00:00, 33.28it/s]


Validation Loss for fold 5: 0.9557713568210602
Final CV Loss: 0.98239985704422
